In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate('crp-db-manager/crp-backend-server-b67475fc5e44.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

In [2]:
# code to add User details to the database
import pandas as pd

user_df = pd.read_csv('crp-db-manager/user_passw.csv')

user_ref = db.collection(u'users')

for index, row in user_df.iterrows():
    doc_ref = user_ref.document(row['uid'])
    
    regno = row['reg-number']
    gendr = row['gender']
    uname = row['name']
    email = row['email']
    phnno = row['phone-number']
    usert = row['user-type']
    
    doc_ref.set({
        u'registrationNumber': regno,
        u'name': uname,
        u'gender': gendr,
        u'email': email,
        u'phoneNumber': phnno,
        u'userType': usert
    })

In [3]:
# code to add Course details to the database
import json

course_ref = db.collection(u'courses')

with open('crp-db-manager/course_parse_data.json', 'r') as course_db:
    course_data = json.loads(course_db.read())
    
    for code, info in course_data.items():
        doc_ref = course_ref.document(str(code))
        try:
            doc_ref.set(info)
        except:
            print(info)

In [ ]:
# Code to fetch details

# Available data: User, Course

doc_ref = db.collection(u'users')

docs = doc_ref.stream()

for doc in docs:
    print(u'{} => {}'.format(doc.id, doc.to_dict()))

In [ ]:
from firebase_admin import auth
import pandas as pd
import bcrypt

user_df = pd.read_csv('crp-db-manager/user_passw.csv')

email_passw = user_df[['email', 'password']]

users = list()

for i in range(len(email_passw)):
    passw = email_passw.iloc[i]['password'].encode()
    salt = bcrypt.gensalt()
    hashed = bcrypt.hashpw(passw, salt)
    users.append(
        auth.ImportUserRecord(
            uid='uid' + str(i + 1),
            email=email_passw.iloc[i]['email'],
            password_hash=hashed,
            password_salt=salt
        )
    )

users

In [ ]:
hash_alg = auth.UserImportHash.bcrypt()

try:
    result = auth.import_users(users, hash_alg=hash_alg)
    for err in result.errors:
        print('Failed to import user:', err.reason)
except exceptions.FirebaseError as error:
    print('Error importing users:', error)


In [5]:
import pandas as pd

user_df = pd.read_csv('crp-db-manager/user_passw.csv')

user_course_list = list()

for i in range(user_df.shape[0]):
    user_course_list.append(user_df.iloc[i]['reg-number'][8:11])

user_df = user_df.assign(Dept = user_course_list)

In [ ]:
from collections import defaultdict
import json

course_dict = defaultdict(list)

courses = dict()
with open('crp-db-manager/course_parse_data.json', 'r') as course_db:
    courses = json.loads(course_db.read())

for elem in courses.keys():
    course_dict[elem[2:5]].append(elem)

course_dict

In [ ]:
import random

student_details = dict()

for i in range(user_df.shape[0]):
    if user_df.iloc[i]['User Type'] == 'STUDENT':
        if user_df.iloc[i]['Dept'] != 'CIV': 
            course_list = course_dict[user_df.iloc[i]['Dept']] + course_dict['HUM'] + course_dict['MAT']
        else:
            course_list = course_dict['HUM'] + course_dict['MAT']

        total_course = random.randint(0, min(7, len(course_list)))
        random_courses = random.sample(course_list, total_course)

        course_rem_set = list(set(course_list) - set(random_courses))

        course_rem = random.sample(course_rem_set, min(10 - total_course, len(course_rem_set)))
        
        student_details[user_df.iloc[i]['Registration Number']] = {
            'rollNumber': user_df.iloc[i]['Registration Number'],
            'reviews': [],
            'questions': [],
            'coursesCompleted': random_courses,
            'coursesRemaining': course_rem
        }

In [ ]:
student_ref = db.collection(u'Student')

for rollNum, info in student_details.items():
    doc_ref = student_ref.document(str(rollNum))
    try:
        doc_ref.set(info)
    except:
        print(info)

In [ ]:
from collections import defaultdict

subscribe_course_set = set()

for elem in student_details.values():
    subscribe_course_set.update(elem.get('coursesCompleted'))
    subscribe_course_set.update(elem.get('coursesRemaining'))

course_count = defaultdict(int)

for elem in subscribe_course_set:
    course_count[elem[2:5]] += 1

faculty_count = defaultdict(int)
for i in range(user_df.shape[0]):
    
    if user_df.iloc[i]['User Type'] in ['FACULTY', 'MENTOR']:
        faculty_count[user_df.iloc[i]['Registration Number'][8:11]] += 1

In [30]:
student_details = dict()

for rows in user_df.iterrows():
    if rows[1]['user-type'] == 'STUDENT':
        student_details[rows[1]['uid']] = {
            'rollNumber': rows[1]['reg-number'],
            'reviews': {},
            'questions': {},
            'coursesCompleted': [],
            'coursesRemaining': []
        }

student_details

{'uid1': {'rollNumber': 'CB.EN.U4CSE20000',
  'reviews': {},
  'questions': {},
  'coursesCompleted': [],
  'coursesRemaining': []},
 'uid2': {'rollNumber': 'CB.EN.U4CSE20001',
  'reviews': {},
  'questions': {},
  'coursesCompleted': [],
  'coursesRemaining': []},
 'uid3': {'rollNumber': 'CB.EN.U4CSE20002',
  'reviews': {},
  'questions': {},
  'coursesCompleted': [],
  'coursesRemaining': []},
 'uid4': {'rollNumber': 'CB.EN.U4CSE20003',
  'reviews': {},
  'questions': {},
  'coursesCompleted': [],
  'coursesRemaining': []}}

In [31]:
student_ref = db.collection(u'students')

for uid, info in student_details.items():
    doc_ref = student_ref.document(str(uid))
    try:
        doc_ref.set(info)
    except:
        print(info)

In [32]:
staff_details = dict()

for rows in user_df.iterrows():
    if rows[1]['user-type'] != 'STUDENT':
        staff_details[rows[1]['uid']] = {
            'regNumber': rows[1]['reg-number'],
            'questions': {},
            'previousCourses': {},
            'currentCourses': {}
        }

staff_details

{'uid5': {'regNumber': 'CB.EN.FTCSE20000',
  'questions': {},
  'previousCourses': {},
  'currentCourses': {}},
 'uid6': {'regNumber': 'CB.EN.FTCSE20001',
  'questions': {},
  'previousCourses': {},
  'currentCourses': {}},
 'uid7': {'regNumber': 'CB.EN.FTCSE20002',
  'questions': {},
  'previousCourses': {},
  'currentCourses': {}}}

In [33]:
staff_ref = db.collection(u'staffs')

for uid, info in staff_details.items():
    doc_ref = staff_ref.document(str(uid))
    try:
        doc_ref.set(info)
    except:
        print(info)

In [23]:
reviews = {
    'reviewId': "",
    'courseCode': "",
    'uid': "",
    'rating': {},
    'reviewText': "",
}

In [22]:
questions = {
    'questionId': "",
    'courseCode': "",
    'uidAsked': "",
    'uidAnswered': "",
    'questionText': "",
    'answerText': ""
}

In [28]:
ratingQuestions = { "0": { 'questionNumber': "0", 'questionText': "Overall rating" } }                                          

In [29]:
rate_ques_ref = db.collection(u'ratingQuestions')

for qid, info in ratingQuestions.items():
    doc_ref = rate_ques_ref.document(str(qid))
    try:
        doc_ref.set(info)
    except:
        print(info)